In [3]:
import json
with open('configuration/aml_workspace.json', 'r') as f:
    aml_config = json.load(f)

subscription_id = aml_config['subscription_id']
resource_group = aml_config['resource_group']
workspace_name = aml_config['workspace_name']
datastore_name = aml_config['datastore_name']
path_on_datastore = aml_config['path_on_datastore']
computename = aml_config['computename']

In [4]:
from azure.ai.ml import MLClient
from azure.ai.ml._restclient.dataset_dataplane.models import Dataset
from azure.identity import (
    AzureCliCredential,
    InteractiveBrowserCredential
)
try:
    # Use AzureCLICredential instead of DefaultAzureCredential since VisualStudioCodeCredential might cause errors.
    # Requires previously logging in to Azure via "az login", and will use the CLI's currently logged in identity.
    credential = AzureCliCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception:
    # Fall back to InteractiveBrowserCredential in case AzureCliCredential not work
    credential = InteractiveBrowserCredential()

# Note that MLCLient is lazy and does not yet connect to Azure here.
# In case the CLI authentication failed, a browser window will pop up later and will require authentication.
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

from azure.ai.ml import command
from azure.ai.ml.entities import Data, Workspace, Environment, BuildContext
from azure.ai.ml import Input, Output
from azure.ai.ml.constants import AssetTypes

Class SystemCreatedStorageAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class SystemCreatedAcrAccount: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class RegistryOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [ ]:
# shutil.copy(config_src, context_dir)

env_docker_context = Environment(
    build=BuildContext(path="sordiai-build-context"),
    name="sordiaiyolov5",
    description="Environment created from a Docker context.",
)
res = ml_client.environments.create_or_update(env_docker_context)

In [ ]:
my_job_inputs = {
    "input_data": Input(type=AssetTypes.URI_FOLDER, path="azureml://datastores/sordiai/paths/")
}
job = command(
    code="./src",
    command="./run.sh ${{inputs.input_data}}",
    inputs=my_job_inputs,
    environment="sordiaiyolov5:1",
    compute='Standard-NC4as-T4-v3',

)
returned_job = ml_client.create_or_update(job)
print(returned_job.services["Studio"].endpoint)

In [ ]:
my_job_inputs = {
    "input_data": Input(type=AssetTypes.URI_FOLDER, path="azureml://datastores/sordiai/paths/sordi_stage_2")
}
my_job_outputs = {
    "yolo_data": Output(type=AssetTypes.URI_FOLDER, path="azureml://datastores/sordiai/paths/output/")
}
job = command(
    code="./src",
 #   command="python sordiai.py --input_data ${{inputs.input_data}} --yolo_data ${{outputs.yolo_data}}",
    command="python sordiai.py --input_data ${{inputs.input_data}}",
    inputs=my_job_inputs,
 #   outputs=my_job_outputs,
    environment="sordiaiyolov5:1",
    compute='Standard-L8s-v2',
)
returned_job = ml_client.create_or_update(job)
print(returned_job.services["Studio"].endpoint)